In [ ]:
#載入套件
import cv2
import numpy as np
import pandas as pd
import datetime 
from matplotlib import pyplot as plt
%matplotlib inline

#自訂義函數
def CurveRecognize(Hue,Saturation,Value,GrayFlag):
    HSV=cv2.cvtColor(Image,cv2.COLOR_BGR2HSV)
    
    LowerBound=np.array([Hue[0],Saturation[0],Value[0]])
    UpperBound=np.array([Hue[1],Saturation[1],Value[1]])
    mask=cv2.inRange(HSV,LowerBound,UpperBound)
    
    Curve=cv2.bitwise_and(Image,Image,mask=mask)
    if GrayFlag == 1:
        Curve=cv2.cvtColor(Curve,cv2.COLOR_BGR2GRAY)
        Curve=np.piecewise(Curve,[Curve<=0,Curve>0],[0,255])
        Curve=Curve.astype('uint8')
        return(Curve)
    else:
        return(mask)
    
def Eraser(Image,Location=[250,0],Shape=[30,625]):
    Filter=Image.copy()
    Filter[
        min(Location[0],Filter.shape[0]):min(Location[0]+Shape[0]+1,Filter.shape[0]),
        min(Location[1],Filter.shape[1]):min(Location[1]+Shape[1]+1,Filter.shape[1])
    ]=0
    
    return(Filter)

def LockCurve(Image):
    Location=np.where(Image==255)
    return(
        Image[
            min(Location[0]):max(Location[0])+1,
            min(Location[1]):max(Location[1])+1
        ]
    )

Period=[20161021,20201020]
#B:Generate the series of dates
def GenerateDateSeries(Period):
    Start=str(Period[0])
    Start=f"{Start[:4]}-{Start[4:6]}-{Start[6:8]}"
    
    End=str(Period[1])
    End=f"{End[:4]}-{End[4:6]}-{End[6:8]}"
    
    DateSeries=[]
    for entry in pd.date_range(Start,End):
        DateSeries.append(entry.strftime("%Y%m%d"))
    
    return(DateSeries)

DateSeries=GenerateDateSeries(Period)
#E:Generate the series of dates

#讀取圖檔
Image=cv2.imread('C:\\python\\imone.png')

#定義綠色上下界
Hue=[35,77]
Saturation=[43,255]
Value=[46,255]

Filter=CurveRecognize(Hue, Saturation,Value,0)
ROI=Filter[100:800,0:1830]
cv2.imshow('curve',ROI)
cv2.imwrite('C:\\python\\green.png', ROI)
cv2.waitKey()
cv2.destroyAllWindows()

def find_bundle(ROI):
    start = []
    end = []
    for i in range(ROI.shape[0]):
        for j in range(ROI.shape[1]):
            if ROI[i,j] == 255:
                start.append(i)
                start.append(j)
    for i in range(ROI.shape[0]-1,0,-1):
        for j in range(ROI.shape[1]-1,0,-1):
            if ROI[i,j] == 255:
                end.append(i)
                end.append(j)
    return ROI[start[0]:end[0],start[1]:end[1]]

ROI = find_bundle(ROI)

def output(x,y,ROI):
    grid = int(ROI.shape[1]/x)
    output_y = []
    ya = y[0]
    yb = y[1]
    xa = 0
    xb = ROI.shape[0]
    for i in range(x):
        for j in range(ROI.shape[0]):
            if ROI[j,i*grid] == 255:
                temp = ROI.shape[0] - j
                output_y.append(ya + (yb - ya)*(temp - xa)/(xb - xa))
                break
    return output_y

y = output(104,[335.84,1275.84],ROI)
plt.plot([i for i in range(104)],y)